# Classification in Python using pyjeo and sklearn

In [ ]:
from IPython.display import display
import geopandas as gpd

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pyjeo as pj
import pandas as pd
import geopandas as gpd

## Create reference data for training

In [ ]:
reference = pj.JimVect('/storage/kempepi/repositories/jeolib-pyjeo/tests/data/modis_ndvi_training.sqlite')
jim = pj.Jim('/storage/kempepi/repositories/jeolib-pyjeo/tests/data/modis_ndvi_2010.tif', band2plane=True)

In [ ]:
dates = [datetime.strptime('01-' + str(month) + '-2010', "%d-%m-%Y") for month in range(1, 13)]
jim.properties.setDimension({'band': ['NDVI'], 'plane': dates})

In [ ]:
jim.xr()

In [ ]:
jim.xr().NDVI.plot(col='time', col_wrap=6)

In [ ]:
pd.DataFrame(reference.dict())

In [ ]:
featurevect = pj.geometry.extract(reference, jim, rule=['allpoints'],
                                  output='/vsimem/features.sqlite',
                                  oformat='SQLite',
                                  co=['OVERWRITE=YES'],
                                  classes=[1, 2],
                                  copy='label')
gdf = gpd.read_file('/vsimem/features.sqlite')
gdf

In [ ]:
plt.figure(figsize=(16, 8))
ax = plt.subplot()
jim.xr().NDVI.isel(time = 0).plot(ax = ax)
gdf.plot(column = 'label', ax = ax, legend = True, categorical=True, cmap='Set1')

In [ ]:
pd.DataFrame(featurevect.dict())

## Train the model

In [ ]:
x = featurevect.np()[:, 1:]
y = featurevect.np()[:, 0:1]
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.33,
                                                    random_state=42)
rfModel = RandomForestClassifier(n_estimators=100,
                                 max_depth=9,
                                 min_samples_leaf=5,
                                 min_samples_split=3,
                                 criterion='gini')
rfModel.fit(x_train, y_train.ravel())

In [ ]:
y_predict = rfModel.predict(x_test)
print(confusion_matrix(y_test, y_predict))
print('accuracy score: {}'.format(accuracy_score(y_test, y_predict)))

## Prediction

In [ ]:
x = jim.np()
x = x.reshape(jim.properties.nrOfPlane(), jim.properties.nrOfRow() * \
                      jim.properties.nrOfCol()).T

jim_class = pj.Jim(ncol=jim.properties.nrOfCol(),
                   nrow=jim.properties.nrOfRow(),
                   otype='Byte')
jim_class.properties.copyGeoReference(jim)
jim_class.np()[:] = rfModel.predict(x).astype(np.dtype(np.uint8)).\
            reshape(jim.properties.nrOfRow(), jim.properties.nrOfCol())
jim_class.properties.setDimension(['water'], 'band')

In [ ]:
plt.figure(figsize=(20, 10))
ax1 = plt.subplot(121)
jim.xr().NDVI.isel(time = 0).plot(ax = ax1)
ax2 = plt.subplot(122)
jim_class.xr().water.plot(cmap = 'Set2', levels = [1, 2], ax = ax2)

# Exercise
Use a single feature to train the classifier (e.g., month of June only)  
Check the accuracy

What is the accuracy?

# Exercise
Replace the Random Forest with a Support Vector Machine
(hint: use the `preprocessing.MinMaxScaler` to scale the input data)

In [ ]:
from sklearn.svm import SVC
from sklearn import preprocessing

What is the accuracy?

In [ ]:
y_predict = svmModel.predict(preprocessing.MinMaxScaler().fit_transform(x_test))
print(confusion_matrix(y_test, y_predict))
print('accuracy score: {}'.format(accuracy_score(y_test, y_predict)))

Prediction

In [ ]:
x = jim.np()
x = x.reshape(jim.properties.nrOfPlane(), jim.properties.nrOfRow() * \
              jim.properties.nrOfCol()).T


jim_class = pj.Jim(ncol=jim.properties.nrOfCol(),
                   nrow=jim.properties.nrOfRow(), otype='Byte')
jim_class.properties.copyGeoReference(jim)
jim_class.np()[:] = 
jim_class.properties.setDimension(['water'], 'band')

Plot

In [ ]:
plt.figure(figsize=(20, 10))
ax1 = plt.subplot(121)
jim.xr().NDVI.isel(time = 0).plot(ax = ax1)
ax2 = plt.subplot(122)
jim_class.xr().water.plot(cmap = 'Set2', levels = [1, 2], ax = ax2)